In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import PIL
from PIL import Image
import cv2
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import time

from IPython import display

In [2]:
def get_images(link_to_dataset):
    IMG_WIDTH=28
    IMG_HEIGHT=28
    img_data_2D =[]
    class_ =[]
    nd_image_array =[]

    for folder in os.listdir(link_to_dataset):
        for file in os.listdir(os.path.join(link_to_dataset,folder)):
            image_path = os.path.join(link_to_dataset, folder, file)
            image = Image.open(image_path).resize((IMG_WIDTH,IMG_HEIGHT))
            # convert image to numpy array
            image_data = np.array(image)
            image = np.array(image)
            flat_image= image.flatten()
            #image = image.astype('float32')
            
            class_.append(folder)
            img_data_2D.append(flat_image)
            nd_image_array.append(image_data)

    return class_, img_data_2D, nd_image_array

In [3]:
class_names_, img_data_, images_array = get_images(r'./Brain')



In [ ]:
# X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y_cat_, test_size=0.30, random_state=32)

In [4]:
X = np.array(images_array)
X = X.astype("float32") / 255.0
X = np.reshape(X, (-1, 28, 28, 1))
X.shape

(410, 28, 28, 1)

In [5]:
y_ = np.array(class_names_)
le = preprocessing.LabelEncoder()
y = le.fit_transform(y_)
np.unique(y)

array([0, 1], dtype=int64)

In [6]:
y_cat_ = tf.keras.utils.to_categorical(y, 2)

In [8]:
batch_size = 25
dataset = tf.data.Dataset.from_tensor_slices((X))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [9]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(500,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

In [10]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='valid',
                                     input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='valid'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [16]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [12]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


In [13]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [14]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [17]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [24]:
EPOCHS = 50
BUFFER_SIZE = 60000
BATCH_SIZE = 25
noise_dim = 500
num_examples_to_generate = 1
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [19]:

#@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      
      fake_output = discriminator(generated_images, training=True)
      

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    

In [20]:
def generate_and_save_images(model, epoch, test_input):

  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(10, 10))

  for i in range(predictions.shape[0]):
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

      plt.savefig('gan_images/image_at_epoch_{:04d}.jpeg'.format(epoch))
  

In [21]:

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    
    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,epoch + 1,seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,epochs, seed)
  


In [ ]:
train(dataset, EPOCHS)

In [132]:
dataset

<BatchDataset shapes: (None, 28, 28, 1), types: tf.float32>